In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm
from pymc.Matplot import plot as pmplot

sns.set()

In [ ]:
dick = pd.read_pickle('country_data.pkl')

def strip(x):
    return x.replace('\t','')

country_populations = pd.read_csv('world_pop.csv',
                                  sep=';',header=None,index_col=0,names=['population'],
                                 thousands=',',converters={0 : strip})

def process_country(country_name):
    
    country = dick[country_name]
    country_population = country_populations.loc[country_name,'population']
    country['conf_per_M'] = country['confirmed'] / (country_population / 1e6)
    country['dead_per_M'] = country['deceased'] / (country_population / 1e6)
    
    return country



In [ ]:
r_prior = pm.rlognormal(0.6, 1 / 0.1 ** 2,100000)
f_prior = pm.rbeta(5,200,100000)

plt.hist(r_prior)

In [ ]:
### ASSUMPTIONS ###
### 1) nr of dead inc is a function of nr of confirmed
### 2) D = f * C
### 3) D = C * r * f 

def inference(df,C,D,country_name):
   
    alpha=10

    #f_alpha = 5
    #f_beta = 100
    
    f_alpha = pm.Uniform('f_alpha',1,100)
    f_beta = pm.Uniform ('f_beta',50,500)
    
    r_mu = pm.Uniform('r_mu',0.05,0.6)
    r_sigma = pm.Uniform('r_sigma',0,0.2)
    
    r = pm.Lognormal('r',r_mu, 1 / r_sigma ** 2)
    
    #r = pm.Lognormal('r',0.27,1/0.05**2)
    
    f = pm.Beta('f',f_alpha,f_beta)
    #m = pm.Uniform ('m',-2,2)

    @pm.deterministic()
    def prediction(C=C,r=r,f=f):
        return C * r * f 

    obs = pm.Poisson('obs',prediction, observed=True,value=D)

    model = pm.Model([r_mu, r_sigma,f_alpha,f_beta,f,r,prediction,obs])
    try:
        map_ = pm.MAP(model)
        map_.fit()
        
    except:
        print ('cant fit {}'.format(country_name))

    mcmc = pm.MCMC(model)

    mcmc.sample(500000,100000,4)

    r_mu_post = mcmc.trace('r_mu')[:]
    r_sigma_post = mcmc.trace('r_sigma')[:]
    r_post = mcmc.trace('r')[:]
    f_post = mcmc.trace('f')[:]
    f_alpha_post = mcmc.trace('f_alpha')[:]
    f_beta_post = mcmc.trace('f_beta')[:]
    
    
    pmplot(r_mu_post,'r_mu_post')
    pmplot(r_sigma_post,'r_sigma_post')
    pmplot(f_alpha_post,'f_alpha_post')
    pmplot(f_beta_post,'f_beta_post')
    pmplot(r_post,'r_post')
    pmplot(f_post,'f_post')
    
    prediction_post = mcmc.trace('prediction')[:,0]

    result = pd.DataFrame({'r_post':r_post,
                          'f_post':f_post,
                          'prediction_post':prediction_post})

    print ()
    print ('++++++++++++++ {} ++++++++++++++++'.format(country_name))
    print (result.describe())
    print()
    
    plt.figure(figsize=(18,12))
    plt.hist(r_prior,color='r',label='r prior',alpha=0.5,histtype='stepfilled',density=True)
    plt.hist(result.r_post,color='g',label='r posterior',alpha=0.7,histtype='stepfilled',density=True)
    plt.legend(loc='upper left')

    plt.figure(figsize=(18,12))
    plt.hist(f_prior,color='r',label='f prior',alpha=0.5,histtype='stepfilled',density=True)
    plt.hist(result.f_post,color='g',label='f posterior',alpha=0.7,histtype='stepfilled',density=True)
    
    plt.legend(loc='upper left')
    
    return result



In [ ]:
def create_posterior_samples(result):
    N = 10000
    f_samples = np.random.choice(result.f_post,replace=True,size=N)
    r_samples = np.random.choice(result.r_post,replace=True,size=N)

    samples = np.array([pm.rpoisson(C[i] * r_samples * f_samples,N) for i in range(len(C))])

    samples.shape

    sample_df = pd.DataFrame(samples,index=range(len(samples)))
    sample_df.index.name='day'
    sample_df.columns.name='sample_nr'
    return sample_df

In [ ]:

def plot(result,df,sample_df,country_name):
    
    samples_89 = np.percentile(sample_df.iloc[-1,:],[5.5,94.5])
    samples_daily_mean = sample_df.mean(axis=1)

    
    plt.figure(figsize=(18,12))
    title_postfix = r' Sample posterior mean growth factor $\mu$: {:.2f} posterior mean mortality rate $\mu$: {:.4f}'.format(
    result.r_post.mean(),result.f_post.mean())
    
    title_postfix_2 = ' Last Day Sample 89% CI: {:.2f} , {:.2f} : Last Week Actual: {}'.format(
        samples_89[0],samples_89[1],df.iloc[-1,2])


    sns.violinplot(data=sample_df.T,inner='quartile',scale='count',palette=['lime'])
    plt.plot(range(len(df)),samples_daily_mean,color='crimson',ls='dashed',label='death samples weekly mean ')


    plt.title('Bayesian Posterior Expected number of deaths per week {}, after the day CONFIRMED hit {} \n'.format(
        country_name.upper(),limit) + title_postfix + '\n' + title_postfix_2)

    plt.ylabel('Expected number of deaths per week')
    plt.xlabel('Week after number of confirmed hit {}'.format(limit))
    plt.scatter(range(len(df)),df['dead_inc'],color='crimson',label='actual data')
    plt.legend(loc='upper left')
    
    plt.savefig('{}_violinplot_weekly.jpg'.format(country_name),format='jpg')
    

In [ ]:
countries = ['Sweden','US','India','Italy',
             'Korea, South','Netherlands','Canada','Germany',
             'United Kingdom','France','Belgium','Austria','Norway','Denmark','Finland']


#countries = ['Finland','Norway','Denmark','Iceland','Australia','Russia','France','United Kingdom',
            #'Japan','Hong Kong']

countries = ['Canada','United Kingdom','Australia','Russia']


for country_name in countries:
    
    df = process_country(country_name)

    limit = 1000
    mask = df['confirmed'] >= limit
    df = df[mask]

    #df = df[-10:]
    
    resample_period = ['W-MON','W-TUE','W-WED','W-THU','W-FRI','W-SAT','W-SUN']
    today = pd.datetime.today().weekday()

    yesterday = lambda i : resample_period[i % 7 -1] if i != 0 else resample_period[6]

    df = df.resample(yesterday(today)).agg({'confirmed' : np.mean,'inc': np.sum,'dead_inc':np.sum,'deceased':np.mean})
    D = df['dead_inc'].resample(yesterday(today)).sum().astype(int)
    C = df['confirmed'].resample(yesterday(today)).mean().astype(int)
    #C = df['inc'].resample(yesterday(today)).mean().astype(int)

    
    print()
    print ('+++++++++++++ {} +++++++++++++'.format(country_name))
    print (df)


    result = inference(df,C,D,country_name)
    sample_df = create_posterior_samples(result)
    #print (sample_df.head(40))
    sample_df.iloc[-1].describe()
    plot(result,df,sample_df,country_name)

In [ ]:
US = pd.read_pickle('us_state_dict.pkl')

In [ ]:
ny = US['New York']
ny.index = pd.DatetimeIndex(ny.index)

In [ ]:
resample_period = ['W-MON','W-TUE','W-WED','W-THU','W-FRI','W-SAT','W-SUN']
today = pd.datetime.today().weekday()

yesterday = lambda i : resample_period[i % 7 -1] if i != 0 else resample_period[6]

ny = ny.resample(yesterday(today)).agg({'confirmed' : np.mean,'inc':np.sum,'dead_inc':np.sum,'deceased':np.mean})
D = ny['dead_inc'].resample(yesterday(today)).sum().astype(int)
C = ny['confirmed'].resample(yesterday(today)).mean().astype(int)    
#C = ny['inc'].resample(yesterday(today)).mean().astype(int)    

#D = ny['dead_inc']
#C = ny['confirmed']

#print (D)

result = inference(ny,C,D,'New York')
sample_df = create_posterior_samples(result)
plot(result,ny,sample_df,'New York')